In [1]:
import pandas as pd
import json
import requests
import time
import re
from bs4 import BeautifulSoup

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}

In [62]:
craigslist = pd.read_csv("craiglistOutputFinal.csv")
craigslist = craigslist.sort_values('CarYear', ascending=False)
craigslist.head()

,CarMake,CarModel,CarYear,CarPrice,CarCondition,MilesOnCar,PostingDate,PostingUrl,CarImageLinks,PostingRegion
457,Mitsubishi,Eclipse Cross,2022,27500.0,new,10,2021-05-31,https://kpr.craigslist.org/ctd/d/kennewick-202...,['https://images.craigslist.org/00G0G_iwxsZJtn...,Kennewick-Pasco-Richland
2699,Chevrolet,Suburban High Country,2021,-1.0,NaN,4248,2021-05-31,https://spokane.craigslist.org/ctd/d/otis-orch...,['https://images.craigslist.org/00Q0Q_aECyW6g1...,Spokane / Coeur d'Alene
2054,Nissan,Versa,2021,16950.0,excellent,4373,2021-05-31,https://seattle.craigslist.org/tac/ctd/d/tacom...,['https://images.craigslist.org/00X0X_laewhzXy...,Seattle-Tacoma
555,Chevrolet,Equinox Lt,2021,33639.0,excellent,4,2021-05-29,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/00w0w_5zmhxECF...,Kennewick-Pasco-Richland
2617,Chevrolet,Lifted Silverado 1500,2021,74999.0,NaN,467,2021-05-31,https://spokane.craigslist.org/ctd/d/spokane-l...,['https://images.craigslist.org/00g0g_gktrvbbL...,Spokane / Coeur d'Alene


In [63]:
craigslist.shape

(3596, 10)

In [57]:
# all unique makesa
makes = craigslist.CarMake.unique()
makes

array(['Mitsubishi', 'Chevrolet', 'Nissan', 'Toyota', 'Honda', 'Subaru',
       'Hyundai', 'Gmc', 'Chrysler', 'Ford', 'Bmw', 'Dodge', 'Jeep',
       'Ram', 'Porsche', 'Land Rover', 'Cadillac', 'Acura', 'Lexus',
       'Kia', 'Buick', nan, 'Volkswagen', 'Genesis', 'Tesla', 'Audi',
       'Volvo', 'Mercedes', 'Mazda', 'Infiniti', 'Alfa Romeo', 'Mini',
       'Fiat', 'Jaguar', 'Maserati', 'Scion', 'Ferrari', 'Mercury',
       'Saturn', 'Suzuki', 'Lincoln', 'Pontiac', 'Hummer', 'Saab',
       'Oldsmobile', 'Isuzu', 'Saleen', 'Plymouth', 'Mack Trucks',
       'Triumph', 'Datsun', 'Studebaker'], dtype=object)

In [5]:
models = craigslist.CarModel.unique()
models

array(['Eclipse Cross', 'Suburban High Country', 'Versa', ...,
       'Truck Rat Rod', '‘32 5 Window Coupe', 'Sport Coupe'], dtype=object)

In [6]:
years = craigslist.CarYear.unique()
years

array([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012,
       2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002, 2001,
       2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991, 1990,
       1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981, 1980, 1979,
       1978, 1977, 1976, 1975, 1974, 1973, 1972, 1971, 1970, 1969, 1968,
       1967, 1966, 1965, 1964, 1963, 1961, 1960, 1959, 1953, 1952, 1951,
       1948, 1946, 1941, 1936, 1935, 1934, 1932, 1929, 1928,   -1],
      dtype=int64)

In [7]:
make_model_year = {}
for make in makes:
    make_model_year[make] = {}
    
    craigslist_slice = craigslist.loc[craigslist.CarMake == make, ["CarModel", "CarYear"]]
    
    for year in craigslist_slice.CarYear.unique():
        temp = craigslist_slice.loc[craigslist.CarYear == year, "CarModel"].values
        make_model_year[make][year] =temp

In [8]:
make_model_year["Mitsubishi"]

{2022: array(['Eclipse Cross'], dtype=object),
 2020: array(['Outlander'], dtype=object),
 2018: array(['Eclipse', 'Outlander Phev Sel', 'Outlander'], dtype=object),
 2014: array(['Outlander Sport Se'], dtype=object),
 2013: array(['Outlander Sport'], dtype=object),
 2012: array(['Outlander Sport Es 4D'], dtype=object),
 2008: array(['Lancer Gts'], dtype=object),
 2005: array([nan], dtype=object),
 2003: array(['Eclipse'], dtype=object)}

In [9]:
req = requests.get("https://www.carqueryapi.com/api/0.3/?callback=?&cmd=getModels&make=mitsubishi&year=2021", headers=headers).content

In [10]:
import difflib
def find_name(craigslist_names, database_names, year):
    ans = []
    
    for name in craigslist_names:
        name_2 = str(name).capitalize()
        guess = difflib.get_close_matches(name_2, database_names, n=1, cutoff=.5)
        if len(guess) == 0:
            guess = ["NA"]
        print((name, guess[0], year))
        ans.append((name, guess[0], year))
    return ans

In [15]:
url = "https://www.carqueryapi.com/api/0.3/?callback=?&cmd=getModels&make={}&year={}"
answer = []
for make in make_model_year:
    print(make)
    for year in make_model_year[make]:
        if year > 2021 or year < 1941:
            continue
        # ordered like so: 
        # dict[make][year] = [(model raw, model db)]
        models = make_model_year[make][year]
        req = requests.get(url.format(make,year), headers=headers).content
        if len(req) == 0:
            answer.extend(find_name(models, ["NA"], year))
            continue
        req = json.loads(req[2:-2])
        model_names = []
        for model_name in req["Models"]:
            model_names.append(model_name["model_name"])
        answer.extend(find_name(models, model_names, year))
        time.sleep(1)
answer

Mitsubishi
('Outlander', 'Outlander', 2020)
('Eclipse', 'NA', 2018)
('Outlander Phev Sel', 'Outlander Sport', 2018)
('Outlander', 'Outlander', 2018)
('Outlander Sport Se', 'Outlander Sport', 2014)
('Outlander Sport', 'Outlander', 2013)
('Outlander Sport Es 4D', 'Outlander', 2012)
('Lancer Gts', 'Lancer', 2008)
(nan, 'Magna', 2005)
('Eclipse', 'Eclipse', 2003)
Chevrolet
('Suburban High Country', 'Suburban', 2021)
('Equinox Lt', 'Equinox', 2021)
('Lifted Silverado 1500', 'Silverado 1500', 2021)
('Silverado 1500 Lt', 'Silverado 1500', 2021)
('Silverado 3500Hd Lt', 'Silverado 3500HD', 2021)
('Silverado 1500 High', 'Silverado 1500', 2021)
('Silverado 1500 High', 'Silverado 1500', 2021)
('Equinox Lt', 'Equinox', 2021)
('Silverado 1500 High', 'Silverado 1500', 2021)
('Trailblazer Ls', 'Traverse', 2021)
('Silverado 2500Hd', 'Silverado 2500HD', 2021)
('Suburban High', 'Suburban', 2021)
('Silverado 3500Hd Lt', 'Silverado 3500HD', 2021)
('Silverado 1500 Lt', 'Silverado 1500', 2021)
('Silverado 25

('Bolt Ev', 'Volt', 2017)
('Silverado 1500 Double', 'Silverado 1500', 2017)
('Colorado Extended Cab', 'Colorado', 2017)
('Colorado Extended Cab', 'Colorado', 2017)
('Chevy Silverado 2500Hd', 'Silverado 2500HD', 2017)
('Silverado Diesels 3500', 'Silverado 3500HD', 2017)
('Silverado Deisel 2500', 'Silverado 2500HD', 2017)
('Colorado Extended Cab', 'Colorado', 2017)
('Equinox', 'Equinox', 2017)
('Colorado Lt 4X4 Gas', 'Colorado', 2017)
('Camaro', 'Camaro', 2017)
('Colorado Z71', 'Colorado', 2017)
('Chevy Malibu', 'Malibu', 2017)
('Silverado 1500 Double', 'Silverado 1500', 2017)
('Silverado 1500 Double', 'Silverado 1500', 2017)
('Silverado 2500 Hd Lt', 'Silverado 2500HD', 2017)
('Silverado Durmax 3500', 'Silverado 3500HD', 2017)
('Chevy Silverado 1500', 'Silverado 1500', 2017)
('Camaro', 'Camaro', 2017)
('Camaro', 'Camaro', 2017)
('Silverado High', 'Silverado 1500', 2017)
('Colorado Extended Cab', 'Colorado', 2017)
('Camaro', 'Camaro', 2017)
('Silverado Duramax Hd', 'Silverado 1500', 2017)

('Silverado 1500 Regular Cab', 'Silverado', 2006)
('Trailblazer', 'TrailBlazer', 2006)
('Silverado 1500', 'Silverado', 2006)
('Tahoe Z71', 'Tahoe', 2006)
('Silverado 1500', 'Silverado', 2005)
('Tahoe', 'Tahoe', 2004)
('Silverado 1500', 'Silverado', 2004)
('Corvette', 'Corvette', 2004)
('Silverado 2500 4X4', 'Silverado', 2004)
('Cavalier', 'Cavalier', 2004)
('Silverado', 'Silverado', 2004)
('Silverado', 'Silverado', 2004)
('Impala', 'Impala', 2004)
('Venture', 'NA', 2003)
('Trailblazer', 'TrailBlazer', 2003)
('Silverado 1500', 'Silverado', 2003)
('Trailblazer', 'TrailBlazer', 2002)
('Cavalier', 'Cavalier', 2002)
('Silverado', 'NA', 2002)
('Camaro', 'Camaro', 2002)
('Silverado 2500Hd', 'NA', 2002)
('Suburban', 'Suburban', 2002)
('Trailblazer', 'TrailBlazer', 2002)
('Prizm', 'Prizm', 2002)
('Suburban', 'Suburban', 2002)
('Camaro', 'Camaro', 2002)
('Tahoe Ls', 'Tahoe', 2001)
('2500Hd Duramax', 'NA', 2001)
('Tracker', 'Tracker', 2001)
('Cavalier', 'Cavalier', 2001)
('Blazer Zr2', 'Blazer', 

('Tacoma Trd Off Road', 'NA', 2019)
('Sienna L Minivan 4D', 'NA', 2019)
('Tacoma Trd Off Road', 'NA', 2019)
('Highlander', 'Highlander', 2019)
('Tundra Double Cab Sr5', 'NA', 2019)
('Tacoma Trd Off Road 4X4', 'NA', 2019)
('Tundra Trd Pro 4X4 Gas', 'NA', 2019)
('Tundra Sr5 Trd Offroad', 'NA', 2019)
('Tacoma', 'Tacoma', 2019)
('Camry Se Fwd Gas Sedan', 'NA', 2019)
('Camry Le Fwd Gas Sedan', 'NA', 2019)
('Sienna L Minivan 4D', 'NA', 2019)
('Tacoma Access Cab Trd', 'NA', 2019)
('Tacoma Double Cab Trd', 'NA', 2019)
('Tacoma Access Cab Trd', 'NA', 2019)
('Tacoma', 'Tacoma', 2019)
('Sienna Le Minivan 4D', 'NA', 2019)
('Lifted 4Runner Sr5 4X4', 'NA', 2019)
('Tacoma', 'Tacoma', 2019)
('Tundra Trd Sport 4X4 Gas', 'NA', 2019)
('4Runner', '4Runner', 2019)
('Tundra Trd Pro 4X4 Gas', 'NA', 2019)
('Sienna Le', 'Sienna', 2019)
('Lifted Tacoma Sr5 4X4 Gas', 'NA', 2019)
('Highlander 4X4 3Rd Row', 'Highlander Hybrid', 2019)
('Rav4 Xle Premium', 'NA', 2019)
('Highlander Le Awd Gas Suv', 'Highlander Hybrid

('4-Runner', '4Runner', 2005)
('Tacoma', 'Tacoma', 2005)
('Tacoma Access Cab Sr5', 'NA', 2005)
('Avalon Limited', 'Avalon', 2005)
('Land Cruiser', 'Land Cruiser', 2005)
('Tacoma Double Cab Sr5', 'NA', 2005)
('Corolla', 'Corolla', 2005)
('Prius Base 4Dr Hatchback', 'NA', 2005)
('Solara', 'Solara', 2005)
('4Runner Limited 4X4', 'NA', 2005)
('4Runner Sr5', '4Runner', 2004)
('Camry', 'Camry', 2004)
('4-Runner', '4Runner', 2004)
('4-Runner', '4Runner', 2004)
('Matrix Xr', 'Mark II', 2004)
('Corolla', 'Corolla', 2004)
('Sequoia', 'Sequoia', 2003)
('Tacoma Double Cab', 'NA', 2003)
('Tundra', 'Tundra', 2003)
('Tacoma', 'NA', 2003)
('Camry Xle', 'Camry', 2003)
('Rav4', 'RAV4', 2002)
('Camry Le', 'Camry', 2002)
('Tacoma', 'NA', 2002)
('Tacoma', 'NA', 2002)
('Rav4', 'RAV4', 2002)
('Tacoma Double Cab', 'NA', 2002)
('Corolla', 'Corolla', 2001)
('Tacoma', 'NA', 2001)
('Tacoma', 'NA', 2001)
('Tacoma Sr5', 'NA', 2001)
('Sequoia', 'Sequoia', 2001)
('Tundra', 'Tundra', 2001)
('Rav4', 'NA', 2001)
('Camry

...

('Renegade Latitude Fwd Gas', 'NA', 2017)
('Renegade', 'Renegade', 2017)
('Renegade', 'Renegade', 2017)
('Wrangler Unlimited Sport', 'Wrangler', 2017)
('Wrangler', 'Wrangler', 2017)
('Wrangler Sport Suv 2D', 'Wrangler', 2017)
('Wrangler Unlimited Sport', 'Wrangler', 2017)
('Wrangler Unlimited Chief', 'Wrangler', 2017)
('Grand Cherokee', 'Grand Cherokee', 2017)
('Wrangler', 'Wrangler', 2017)
('Cherokee', 'Cherokee', 2017)
('Cherokee', 'Cherokee', 2017)
('Wrangler Unlimited', 'Wrangler', 2017)
('Wrangler Unlimited Sport', 'Wrangler', 2017)
('Renegade Latitude Sport', 'Renegade', 2017)
('Wrangler', 'Wrangler', 2017)
('Compass All New Latitude', 'NA', 2017)
('Wrangler', 'Wrangler', 2017)
('Grand Cherokee Altitude 4', 'Grand Cherokee', 2017)
('Cherokee', 'Cherokee', 2017)
('Grand Cherokee Laredo 4Wd', 'Grand Cherokee', 2017)
('Wrangler Unlimited 75Th 4X4', 'NA', 2017)
('Wrangler Unlimited Sport', 'Wrangler', 2017)
('Wrangler', 'Wrangler', 2017)
('Cherokee', 'Cherokee', 2017)
('Wrangler', 'W

('911 Carrera S', 'NA', 2018)
('Cayenne S Awd', 'Cayenne', 2018)
('Macan S', 'Macan', 2018)
('Panamera Turbo', 'Panamera', 2017)
('Macan', 'Macan', 2017)
('Macan', 'Macan', 2017)
('Cayenne Turbo', 'Cayenne', 2017)
('Macan', 'Macan', 2017)
('Cayenne', 'Cayenne', 2016)
('Panamera', 'Panamera', 2016)
('Cayenne Gts', 'Cayenne', 2016)
('Boxster Roadster 2D', 'Boxster', 2015)
('Cayenne S', 'Cayenne', 2011)
('Cayenne S', 'Cayenne', 2011)
('Boxster', 'Boxster', 2003)
('911 Carrera', 'Carrera GT', 2002)
('911 Carrera', 'Carrera GT', 2002)
('Boxter', 'Boxster', 2001)
('911 Carrera', 'Carrera GT', 2001)
('Boxster', 'Boxster', 1999)
Land Rover
('Discovery Sport Se', 'Discovery Sport', 2020)
('Discovery Sport Se', 'Discovery Sport', 2020)
('Range Rover Evoque', 'Range Rover Evoque', 2019)
('Range Rover Velar', 'Range Rover Evoque', 2019)
('Range Rover Evoque', 'Range Rover Evoque', 2018)
('Discovery', 'Discovery Sport', 2017)
('Discovery', 'Discovery Sport', 2017)
('Discovery Sport Se', 'Discovery 

('Lucerne', 'Lucerne', 2011)
('Lucerne Cxl', 'Lucerne', 2011)
('Regal', 'Regal', 2011)
('Enclave', 'Enclave', 2008)
('Enclave Cxl 4Dr Crossover', 'NA', 2008)
('Rainer', 'Rainier', 2006)
('Lesabre', 'LeSabre', 2005)
('Regal', 'Regal', 2004)
('Regal', 'Regal', 2003)
('Lesabre', 'LeSabre', 2001)
('Century', 'Century', 1999)
('Regal', 'Regal', 1998)
('Regal', 'Regal', 1998)
('Park Avenue', 'Park Avenue', 1998)
('Regal', 'Regal', 1993)
('Riviere', 'Riviera', 1988)
('Grand National', 'NA', 1968)
nan
Volkswagen
('Atlas Se W/Tech Pkg', 'NA', 2020)
('Atlas', 'NA', 2019)
('Beetle 2.0T Se', 'Beetle', 2019)
('Golf R 4Motion', 'Golf SportWagen', 2019)
('Atlas Sel Sport', 'NA', 2019)
('Vw Jetta', 'Jetta', 2019)
('Golf Gti S Hatchback', 'NA', 2018)
('Golf Se Fwd Gas Auto', 'NA', 2018)
('Passat 2.0T S Sedan', 'NA', 2018)
('Atlas S 4Motion Sport', 'NA', 2018)
('Atlas S 4Motion Sport', 'NA', 2018)
('Atlas S 4Motion Sport', 'NA', 2018)
('Golf Sportwagen Tsi S', 'Golf SportWagen', 2018)
('Golf Tsi S Hatch

('Mazda6', 'Mazda6', 2016)
('Mazda6 I Grand Touring', 'NA', 2016)
('Mx-5 Miata Grand Touring', 'NA', 2016)
('Mx-5 Miata Grand Touring', 'NA', 2016)
('Mazda6 I Grand Touring', 'NA', 2016)
('Mazda6', 'Mazda6', 2015)
('Cx-5', 'CX-5', 2014)
('Mazda3 I Touring Mt 5-Door', 'NA', 2014)
('Mazda3 I Grand Touring', 'NA', 2014)
('Mazda5', 'NA', 2014)
('Mazda3 I Grand Touring', 'NA', 2013)
('3', '3', 2013)
('5', '5', 2012)
('6', '6', 2012)
('5', '5', 2012)
('3I', '3', 2011)
('3', '3', 2010)
('Mx-5', 'NA', 2008)
('6', '6', 2008)
('Mazda3 I Touring', 'NA', 2006)
('Mazda6 S', 'NA', 2005)
('6', '6', 2004)
('Mazda3 I', 'NA', 2004)
('6', '6', 2003)
('Miata', 'MX-5 Miata', 2001)
('Miata Mx5', 'MX-5 Miata', 1999)
('Miata', 'MX-5 Miata', 1999)
('Protege Es 81K Miles', 'NA', 1995)
('Miata', 'MX-5 Miata', 1995)
('Bongo Diesel Fire', 'NA', 1993)
('323', '323', 1991)
('929', 'NA', 1991)
Infiniti
('Q70 3.7 Luxe Sedan 4D', 'NA', 2019)
('Q50 3.0T Luxe Sedan 4D', 'NA', 2018)
('Q50 3.0T Luxe Sedan 4D', 'NA', 2018)


[('Outlander', 'Outlander', 2020),
 ('Eclipse', 'NA', 2018),
 ('Outlander Phev Sel', 'Outlander Sport', 2018),
 ('Outlander', 'Outlander', 2018),
 ('Outlander Sport Se', 'Outlander Sport', 2014),
 ('Outlander Sport', 'Outlander', 2013),
 ('Outlander Sport Es 4D', 'Outlander', 2012),
 ('Lancer Gts', 'Lancer', 2008),
 (nan, 'Magna', 2005),
 ('Eclipse', 'Eclipse', 2003),
 ('Suburban High Country', 'Suburban', 2021),
 ('Equinox Lt', 'Equinox', 2021),
 ('Lifted Silverado 1500', 'Silverado 1500', 2021),
 ('Silverado 1500 Lt', 'Silverado 1500', 2021),
 ('Silverado 3500Hd Lt', 'Silverado 3500HD', 2021),
 ('Silverado 1500 High', 'Silverado 1500', 2021),
 ('Silverado 1500 High', 'Silverado 1500', 2021),
 ('Equinox Lt', 'Equinox', 2021),
 ('Silverado 1500 High', 'Silverado 1500', 2021),
 ('Trailblazer Ls', 'Traverse', 2021),
 ('Silverado 2500Hd', 'Silverado 2500HD', 2021),
 ('Suburban High', 'Suburban', 2021),
 ('Silverado 3500Hd Lt', 'Silverado 3500HD', 2021),
 ('Silverado 1500 Lt', 'Silverado 1

In [40]:
df2 = pd.DataFrame(answer)
df2.columns = ["RawModel", "GuessModel", "Year"]
df2.to_csv("./raw_to_guess.csv", index=False)
print(df2.shape)
df2

(3528, 3)


,RawModel,GuessModel,Year
0,Outlander,Outlander,2020
1,Eclipse,NA,2018
2,Outlander Phev Sel,Outlander Sport,2018
3,Outlander,Outlander,2018
4,Outlander Sport Se,Outlander Sport,2014
...,...,...,...
3523,Diesel Truck,NA,1987
3524,Tr6,TR6,1975
3525,240Z,240Z,1971
3526,Hawk,NA,1963


In [65]:
df2.columns = ["Model", "alias", "Year"]
df2

,Model,alias,Year
0,Outlander,Outlander,2020
1,Eclipse,NA,2018
2,Outlander Phev Sel,Outlander Sport,2018
3,Outlander,Outlander,2018
4,Outlander Sport Se,Outlander Sport,2014
...,...,...,...
3523,Diesel Truck,NA,1987
3524,Tr6,TR6,1975
3525,240Z,240Z,1971
3526,Hawk,NA,1963


In [72]:
craigslist.shape

(3596, 10)

In [74]:
final_craigslist = pd.merge(craigslist, df2, how="left", left_on=["CarModel", "CarYear"], right_on=["Model", "Year"])
print(final_craigslist.shape)
final_craigslist

(8031, 13)


,CarMake,CarModel,CarYear,CarPrice,CarCondition,MilesOnCar,PostingDate,PostingUrl,CarImageLinks,PostingRegion,Model,alias,Year
0,Mitsubishi,Eclipse Cross,2022,27500.0,new,10,2021-05-31,https://kpr.craigslist.org/ctd/d/kennewick-202...,['https://images.craigslist.org/00G0G_iwxsZJtn...,Kennewick-Pasco-Richland,NaN,NaN,NaN
1,Chevrolet,Suburban High Country,2021,-1.0,NaN,4248,2021-05-31,https://spokane.craigslist.org/ctd/d/otis-orch...,['https://images.craigslist.org/00Q0Q_aECyW6g1...,Spokane / Coeur d'Alene,Suburban High Country,Suburban,2021.0
2,Nissan,Versa,2021,16950.0,excellent,4373,2021-05-31,https://seattle.craigslist.org/tac/ctd/d/tacom...,['https://images.craigslist.org/00X0X_laewhzXy...,Seattle-Tacoma,Versa,Versa,2021.0
3,Chevrolet,Equinox Lt,2021,33639.0,excellent,4,2021-05-29,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/00w0w_5zmhxECF...,Kennewick-Pasco-Richland,Equinox Lt,Equinox,2021.0
4,Chevrolet,Equinox Lt,2021,33639.0,excellent,4,2021-05-29,https://kpr.craigslist.org/ctd/d/college-place...,['https://images.craigslist.org/00w0w_5zmhxECF...,Kennewick-Pasco-Richland,Equinox Lt,Equinox,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8026,NaN,NaN,-1,44880.0,NaN,37343,2021-05-28,https://wenatchee.craigslist.org/ctd/d/wenatch...,['https://images.craigslist.org/00303_eRzKEG1k...,Wenatchee,NaN,NaN,NaN
8027,NaN,NaN,-1,52880.0,NaN,67601,2021-05-27,https://wenatchee.craigslist.org/ctd/d/wenatch...,['https://images.craigslist.org/00z0z_biuvkPDt...,Wenatchee,NaN,NaN,NaN
8028,NaN,NaN,-1,47880.0,NaN,53485,2021-05-31,https://wenatchee.craigslist.org/ctd/d/wenatch...,['https://images.craigslist.org/00s0s_dkDmiD7I...,Wenatchee,NaN,NaN,NaN
8029,NaN,NaN,-1,30880.0,NaN,85275,2021-05-31,https://wenatchee.craigslist.org/ctd/d/wenatch...,['https://images.craigslist.org/01717_52T7u2Cj...,Wenatchee,NaN,NaN,NaN


In [75]:
final_craigslist = final_craigslist[final_craigslist.alias != "NA"]

In [79]:
final_craigslist = final_craigslist.drop_duplicates()

In [80]:
final_craigslist.to_csv("./craigslist_with_alias.csv", index=False)